https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

In [61]:
import glob
import pandas as pd
import numpy as np
import json

In [62]:
annotations=glob.glob('data/OPP-115/annotations/*')

In [63]:
annotations_list=[]
for file in annotations:
    annotations_list.append(pd.read_csv(file, header=None))

In [64]:
annot=pd.concat(annotations_list, ignore_index=True)

In [65]:
annot.columns = ['annotation_ID', 'batch_ID', 'annotator_ID', 'policy_ID', 'segment_ID','category_name',
            'attribute_value_pairs','date','policy_URL']

In [66]:
annot['dict'] = annot.attribute_value_pairs.apply(lambda x: json.loads(x))

In [67]:
annot.head()

,annotation_ID,batch_ID,annotator_ID,policy_ID,segment_ID,category_name,attribute_value_pairs,date,policy_URL,dict
0,13160,test_category_labeling_highlight_fordham_aaaa,121,3828,0,Other,"{""Other Type"": {""endIndexInSegment"": 575, ""sta...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Other Type': {'endIndexInSegment': 575, 'sta..."
1,13161,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
2,13162,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
3,13163,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
4,13164,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."


In [68]:
for x in annot['dict']:
    for key, value in x.items():
        value.pop('endIndexInSegment', None)
        value.pop('startIndexInSegment', None)

In [69]:
text=[]
category=[]
subcat=[]
label=[]
counter=0
for i,x in enumerate(annot['dict']):
    for key, value in x.items():
        subcat.append(key)
        if value.get('selectedText')==None:
            text.append('noSelectedText')
        else:
            text.append(value.get('selectedText'))  
        category.append(annot['category_name'][i])
        for k, v in value.items():
            if k=='value':
                label.append(v)

In [70]:
d=list(zip(text, category, subcat, label))

In [71]:
data=pd.DataFrame(d, columns=['text', 'category', 'subcategory', 'label'])

In [72]:
data.head()

,text,category,subcategory,label
0,"Effective Date: May 7, 2015 Kraft Site Privacy...",Other,Other Type,Introductory/Generic
1,noSelectedText,First Party Collection/Use,Collection Mode,not-selected
2,collec,First Party Collection/Use,Choice Scope,Collection
3,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website
4,"personally-identifiable information, such as",First Party Collection/Use,Personal Information Type,Generic personal information


In [73]:
data['cat_sub'] = data['category'] +'-'+ data['subcategory']

In [74]:
data.groupby('cat_sub').count()

,text,category,subcategory,label
cat_sub,,,,
Data Retention-Personal Information Type,370,370,370,370
Data Retention-Retention Period,370,370,370,370
Data Retention-Retention Purpose,370,370,370,370
Data Security-Security Measure,1008,1008,1008,1008
Do Not Track-Do Not Track policy,90,90,90,90
First Party Collection/Use-Action First-Party,8935,8935,8935,8935
First Party Collection/Use-Choice Scope,8935,8935,8935,8935
First Party Collection/Use-Choice Type,8935,8935,8935,8935
First Party Collection/Use-Collection Mode,8935,8935,8935,8935


In [15]:
data.groupby(['category','subcategory']).nunique().count()

text           36
category       36
subcategory    36
label          36
cat_sub        36
dtype: int64

In [16]:
data['cat_sub_lab'] = data['category'] +'-'+ data['subcategory'] +'-'+ data['label']

In [17]:
import string, re
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
data['text'] = data['text'].apply(lambda x : remove_punct(x.lower()))

data = data[data['text']!='null']
data = data[data['text']!='noselectedtext']
data = data[data['text']!='']
data = data[data['text']!=' ']
data.reset_index(inplace=True, drop=True)
ds = data.drop_duplicates()

In [18]:
ds.shape

(36717, 6)

In [19]:
ds.groupby('category').count()

,text,subcategory,label,cat_sub,cat_sub_lab
category,,,,,
Data Retention,676,676,676,676,676
Data Security,808,808,808,808,808
Do Not Track,67,67,67,67,67
First Party Collection/Use,15346,15346,15346,15346,15346
International and Specific Audiences,582,582,582,582,582
Other,2759,2759,2759,2759,2759
Policy Change,932,932,932,932,932
Third Party Sharing/Collection,10655,10655,10655,10655,10655
"User Access, Edit and Deletion",1094,1094,1094,1094,1094


In [20]:
ds.groupby(['category','subcategory']).count().shape

(36, 4)

In [21]:
ds.groupby(['category','subcategory','label']).count().shape

(259, 3)

In [22]:
ds['tokenized'] = ds['text'].apply(lambda x : re.split(' ', x))
# ds['tokenized_text']=res

/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
from gensim.models import Word2Vec
import logging
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [24]:
# w2v = Word2Vec(ds['tokenized'])
# w2v.save('word2vec.model')

In [25]:
w2v = Word2Vec.load('word2vec.model')

2019-09-19 20:43:21,229 : INFO : loading Word2Vec object from word2vec.model
/miniconda3/envs/tensorflow/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-09-19 20:43:21,286 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2019-09-19 20:43:21,287 : INFO : setting ignored attribute vectors_norm to None
2019-09-19 20:43:21,290 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2019-09-19 20:43:21,295 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2019-09-19 20:43:21,296 : INFO : setting ignored attribute cum_table to None
2019-09-19 20:43:21,298 : INFO : loaded word2vec.model


In [26]:
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()

In [27]:
ds.reset_index(inplace=True, drop=True)

In [28]:
word_vectors = w2v.wv

In [29]:
word2id = {k: v.index for k, v in word_vectors.vocab.items()}

In [30]:
def add_new_word(new_word, new_vector, new_index, embedding_matrix, word2id):
    embedding_matrix = np.insert(embedding_matrix, [new_index], [new_vector], axis=0)
    
    word2id = {word: (index+1) if index >= new_index else index for word, index in word2id.items()}
    word2id[new_word] = new_index
    return embedding_matrix, word2id

UNK_INDEX = 0
UNK_TOKEN = 'UNK'

embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id = add_new_word(UNK_TOKEN, unk_vector, UNK_INDEX, embedding_matrix, word2id)

In [31]:
len(word2id)

3933

In [32]:
embedding_matrix.shape

(3933, 100)

In [33]:
embedding_matrix[0].shape

(100,)

In [34]:
embedding_matrix = np.insert(embedding_matrix, len(embedding_matrix), [np.zeros((100,))], axis=0)

In [35]:
len(embedding_matrix)

3934

In [36]:
embedding_matrix[3933].shape

(100,)

In [37]:
def get_int_data(token_text, word2id):
    x = []
    unk_count = 0
    for item in token_text:
        temp=[]
        x.append(temp)
        for word in item:
            if word in word2id:
                temp.append(word2id.get(word))
            else:
                temp.append(UNK_INDEX)
                unk_count += 1
    print('Data created. Percentage of unknown words: %.3f' % (unk_count))
    return np.array(x)

x=get_int_data(ds.tokenized, word2id)
x.shape

Data created. Percentage of unknown words: 7006.000


(36717,)

In [38]:
ds['enumerated_text']=x

/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
#ds['enumerated_text']=ds['enumerated_text'].apply(lambda x: np.array(x))

In [40]:
target = ds.category

In [41]:
ds['Do_Not_Track'] = ds['category'].apply(lambda x: 1 if x == 'Do Not Track' else 0)
ds['Other'] = ds['category'].apply(lambda x: 1 if x == 'Other' else 0)
ds['First_Party_Collection_Use'] = ds['category'].apply(lambda x: 1 if x == 'First Party Collection/Use' else 0)
ds['Third_Party_Sharing_Collection'] = ds['category'].apply(lambda x: 1 if x == 'Third Party Sharing/Collection' else 0)
ds['User_Choice_Control'] = ds['category'].apply(lambda x: 1 if x == 'User Choice/Control' else 0)
ds['International_and_Specific_Audiences'] = ds['category'].apply(lambda x: 1 if x == 'International and Specific Audiences' else 0)
ds['Data_Security'] = ds['category'].apply(lambda x: 1 if x == 'Data Security' else 0)
ds['Policy_Change'] = ds['category'].apply(lambda x: 1 if x == 'Policy Change' else 0)
ds['Data_Retention'] = ds['category'].apply(lambda x: 1 if x == 'Data Retention' else 0)
ds['User_Access_Edit_and_Deletion'] = ds['category'].apply(lambda x: 1 if x == 'User Access, Edit and Deletion' else 0)

/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [42]:
# sanity check
print(ds['Do_Not_Track'].sum())
print(ds['Other'].sum())
print(ds['First_Party_Collection_Use'].sum())
print(ds['Third_Party_Sharing_Collection'].sum())
print(ds['User_Choice_Control'].sum())
print(ds['International_and_Specific_Audiences'].sum())
print(ds['Data_Security'].sum())
print(ds['Policy_Change'].sum())
print(ds['Data_Retention'].sum())
print(ds['User_Access_Edit_and_Deletion'].sum())
ds.head()

67
2759
15346
10655
3798
582
808
932
676
1094


,text,category,subcategory,label,cat_sub,cat_sub_lab,tokenized,enumerated_text,Do_Not_Track,Other,First_Party_Collection_Use,Third_Party_Sharing_Collection,User_Choice_Control,International_and_Specific_Audiences,Data_Security,Policy_Change,Data_Retention,User_Access_Edit_and_Deletion
0,effective date may 7 2015 kraft site privacy n...,Other,Other Type,Introductory/Generic,Other-Other Type,Other-Other Type-Introductory/Generic,"[effective, date, may, 7, 2015, kraft, site, p...","[553, 244, 11, 2150, 839, 1332, 35, 26, 139, 2...",0,1,0,0,0,0,0,0,0,0
1,collec,First Party Collection/Use,Choice Scope,Collection,First Party Collection/Use-Choice Scope,First Party Collection/Use-Choice Scope-Collec...,[collec],[2151],0,0,1,0,0,0,0,0,0,0
2,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website,First Party Collection/Use-Action First-Party,First Party Collection/Use-Action First-Party-...,"[register, on, our, website, or, participate, ...","[204, 19, 10, 52, 6, 212, 13, 10, 168, 3, 363]",0,0,1,0,0,0,0,0,0,0
3,personallyidentifiable information such as,First Party Collection/Use,Personal Information Type,Generic personal information,First Party Collection/Use-Personal Informatio...,First Party Collection/Use-Personal Informatio...,"[personallyidentifiable, information, such, as]","[630, 5, 31, 23]",0,0,1,0,0,0,0,0,0,0
4,if you choose to register on our website or pa...,First Party Collection/Use,Choice Type,Opt-in,First Party Collection/Use-Choice Type,First Party Collection/Use-Choice Type-Opt-in,"[if, you, choose, to, register, on, our, websi...","[32, 4, 105, 1, 204, 19, 10, 52, 6, 212, 13, 1...",0,0,1,0,0,0,0,0,0,0


In [43]:
category_df = ds[['enumerated_text','Do_Not_Track','Other','First_Party_Collection_Use','Third_Party_Sharing_Collection',\
                  'User_Choice_Control','International_and_Specific_Audiences','Data_Security','Policy_Change','Data_Retention',\
                 'User_Access_Edit_and_Deletion']]
category_df.head()

,enumerated_text,Do_Not_Track,Other,First_Party_Collection_Use,Third_Party_Sharing_Collection,User_Choice_Control,International_and_Specific_Audiences,Data_Security,Policy_Change,Data_Retention,User_Access_Edit_and_Deletion
0,"[553, 244, 11, 2150, 839, 1332, 35, 26, 139, 2...",0,1,0,0,0,0,0,0,0,0
1,[2151],0,0,1,0,0,0,0,0,0,0
2,"[204, 19, 10, 52, 6, 212, 13, 10, 168, 3, 363]",0,0,1,0,0,0,0,0,0,0
3,"[630, 5, 31, 23]",0,0,1,0,0,0,0,0,0,0
4,"[32, 4, 105, 1, 204, 19, 10, 52, 6, 212, 13, 1...",0,0,1,0,0,0,0,0,0,0


In [44]:
from keras.preprocessing.sequence import pad_sequences
max_length = max(category_df.enumerated_text.apply(lambda x: len(x)))
padded_docs = pad_sequences(category_df.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

Using TensorFlow backend.


In [45]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Flatten, Dropout

In [46]:
from keras import regularizers

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Do_Not_Track, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 30.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.05)))
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('do_not_track.h5')

from sklearn.metrics import confusion_matrix, f1_score, recall_score
print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/5
25701/25701 [==============================] - 18s 704us/step - loss: 1.9596 - acc: 0.9961 - val_loss: 0.2857 - val_acc: 0.9986
Epoch 2/5
25701/25701 [==============================] - 18s 694us/step - loss: 0.2804 - acc: 0.9966 - val_loss: 0.4047 - val_acc: 0.9949
Epoch 3/5
25701/25701 [==============================] - 18s 688us/step - loss: 0.1800 - acc: 0.9979 - val_loss: 0.1060 - val_acc: 0.9990
Epoch 4/5
25701/25701 [==============================] - 17s 661us/step - loss: 0.1491 - acc: 0.9983 - val_loss: 0.1327 - val_acc: 0.9984
Epoch 5/5
25701/25701 [==============================] - 17s 655us/step - loss: 0.1299 - acc: 0.9987 - val_loss: 0.0980 - val_acc: 0.9987
[[25644     9]
 [    5    43]]
0.86
0.8958333333333334
[[10991     6]
 [    8    11]]
0.6111111111111113
0.5789473684210527


In [47]:
from keras import regularizers

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Do_Not_Track, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 300.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
# model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('do_not_track1.h5')

from sklearn.metrics import confusion_matrix, f1_score, recall_score
print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/5
25701/25701 [==============================] - 15s 593us/step - loss: 9.0386 - acc: 0.9952 - val_loss: 0.0278 - val_acc: 0.9983
Epoch 2/5
25701/25701 [==============================] - 15s 573us/step - loss: 9.0308 - acc: 0.9981 - val_loss: 0.0278 - val_acc: 0.9983
Epoch 3/5
25701/25701 [==============================] - 15s 579us/step - loss: 9.0308 - acc: 0.9981 - val_loss: 0.0278 - val_acc: 0.9983
Epoch 4/5
25701/25701 [==============================] - 15s 579us/step - loss: 9.0308 - acc: 0.9981 - val_loss: 0.0278 - val_acc: 0.9983
Epoch 5/5
25701/25701 [==============================] - 3s 102us/step - loss: 9.0308 - acc: 0.9981 - val_loss: 0.0278 - val_acc: 0.9983
[[25653     0]
 [   48     0]]


/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0
0.0
[[10997     0]
 [   19     0]]
0.0
0.0


In [48]:
# Other Model
x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Other, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 10.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('other.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 90s 3ms/step - loss: 1.2865 - acc: 0.8170 - val_loss: 0.6962 - val_acc: 0.8379
Epoch 2/10
25701/25701 [==============================] - 17s 681us/step - loss: 0.9249 - acc: 0.8566 - val_loss: 0.7760 - val_acc: 0.8095
Epoch 3/10
25701/25701 [==============================] - 21s 798us/step - loss: 0.8611 - acc: 0.8709 - val_loss: 0.6460 - val_acc: 0.8592
Epoch 4/10
25701/25701 [==============================] - 20s 782us/step - loss: 0.8717 - acc: 0.8830 - val_loss: 0.6931 - val_acc: 0.8773
Epoch 5/10
25701/25701 [==============================] - 17s 661us/step - loss: 0.8041 - acc: 0.8888 - val_loss: 0.6683 - val_acc: 0.8624
Epoch 6/10
25701/25701 [==============================] - 17s 649us/step - loss: 0.7522 - acc: 0.9008 - val_loss: 0.6500 - val_acc: 0.8670
Epoch 7/10
25701/25701 [==============================] - 17s 649us/step - loss: 0.8401 - acc: 0.9148 - val_loss: 0.74

In [49]:
#  First Party Collection/Use Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.First_Party_Collection_Use, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 10.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('first_party_collection_use.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 17s 680us/step - loss: 3.5967 - acc: 0.4708 - val_loss: 1.2694 - val_acc: 0.4494
Epoch 2/10
25701/25701 [==============================] - 17s 658us/step - loss: 1.7244 - acc: 0.5048 - val_loss: 1.0670 - val_acc: 0.5519
Epoch 3/10
25701/25701 [==============================] - 18s 700us/step - loss: 1.6122 - acc: 0.5382 - val_loss: 1.1313 - val_acc: 0.5551
Epoch 4/10
25701/25701 [==============================] - 17s 650us/step - loss: 1.5410 - acc: 0.5681 - val_loss: 1.2711 - val_acc: 0.5655
Epoch 5/10
25701/25701 [==============================] - 17s 648us/step - loss: 1.4917 - acc: 0.5946 - val_loss: 0.9915 - val_acc: 0.6178
Epoch 6/10
25701/25701 [==============================] - 17s 674us/step - loss: 1.4483 - acc: 0.6176 - val_loss: 1.0093 - val_acc: 0.6255
Epoch 7/10
25701/25701 [==============================] - 17s 673us/step - loss: 1.3921 - acc: 0.6361 - val_loss: 0.

In [50]:
#  Data Retention Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Data_Retention, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 40.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('data_retention.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 18s 703us/step - loss: 1.8786 - acc: 0.7662 - val_loss: 0.9764 - val_acc: 0.8736
Epoch 2/10
25701/25701 [==============================] - 17s 668us/step - loss: 1.5957 - acc: 0.8093 - val_loss: 1.1598 - val_acc: 0.8464
Epoch 3/10
25701/25701 [==============================] - 20s 763us/step - loss: 1.5642 - acc: 0.8367 - val_loss: 0.8983 - val_acc: 0.8866
Epoch 4/10
25701/25701 [==============================] - 21s 808us/step - loss: 1.3356 - acc: 0.8602 - val_loss: 1.6610 - val_acc: 0.7780
Epoch 5/10
25701/25701 [==============================] - 15s 590us/step - loss: 1.3650 - acc: 0.8613 - val_loss: 0.9649 - val_acc: 0.8666
Epoch 6/10
25701/25701 [==============================] - 20s 774us/step - loss: 1.2653 - acc: 0.8716 - val_loss: 1.1556 - val_acc: 0.8653
Epoch 7/10
25701/25701 [==============================] - 21s 826us/step - loss: 1.0824 - acc: 0.8855 - val_loss: 0.

In [51]:
#  Data Security Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Data_Security, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 40.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(50, activation='tanh', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('data_security.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 23s 880us/step - loss: 3.2660 - acc: 0.8501 - val_loss: 0.9611 - val_acc: 0.9023
Epoch 2/10
25701/25701 [==============================] - 20s 786us/step - loss: 1.1606 - acc: 0.9045 - val_loss: 1.0616 - val_acc: 0.8203
Epoch 3/10
25701/25701 [==============================] - 19s 751us/step - loss: 0.9385 - acc: 0.9226 - val_loss: 0.6411 - val_acc: 0.9461
Epoch 4/10
25701/25701 [==============================] - 20s 788us/step - loss: 0.8606 - acc: 0.9442 - val_loss: 0.6253 - val_acc: 0.9272
Epoch 5/10
25701/25701 [==============================] - 19s 754us/step - loss: 0.7057 - acc: 0.9499 - val_loss: 0.5148 - val_acc: 0.9713
Epoch 6/10
25701/25701 [==============================] - 21s 802us/step - loss: 0.8601 - acc: 0.9623 - val_loss: 0.5710 - val_acc: 0.9734
Epoch 7/10
25701/25701 [==============================] - 21s 814us/step - loss: 0.6402 - acc: 0.9645 - val_loss: 0.

In [52]:
#  International and Specific Audiences Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.International_and_Specific_Audiences, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 50.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('internation_specific_audiences.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 19s 731us/step - loss: 1.9720 - acc: 0.8891 - val_loss: 1.1745 - val_acc: 0.9102
Epoch 2/10
25701/25701 [==============================] - 17s 672us/step - loss: 1.5172 - acc: 0.9191 - val_loss: 2.5771 - val_acc: 0.8617
Epoch 3/10
25701/25701 [==============================] - 18s 682us/step - loss: 1.9192 - acc: 0.9322 - val_loss: 2.6822 - val_acc: 0.9355
Epoch 4/10
25701/25701 [==============================] - 17s 651us/step - loss: 1.4754 - acc: 0.9508 - val_loss: 1.6216 - val_acc: 0.8824
Epoch 5/10
25701/25701 [==============================] - 17s 653us/step - loss: 1.2988 - acc: 0.9564 - val_loss: 0.9926 - val_acc: 0.9604
Epoch 6/10
25701/25701 [==============================] - 17s 653us/step - loss: 1.1108 - acc: 0.9611 - val_loss: 1.3079 - val_acc: 0.9261
Epoch 7/10
25701/25701 [==============================] - 17s 661us/step - loss: 0.6700 - acc: 0.9712 - val_loss: 1.

In [53]:
#  International and Specific Audiences Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.International_and_Specific_Audiences, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 45.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('internation_specific_audiences1.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 18s 705us/step - loss: 1.8597 - acc: 0.9052 - val_loss: 1.0832 - val_acc: 0.9155
Epoch 2/10
25701/25701 [==============================] - 20s 778us/step - loss: 1.4710 - acc: 0.9274 - val_loss: 2.8183 - val_acc: 0.9447
Epoch 3/10
25701/25701 [==============================] - 18s 692us/step - loss: 1.3554 - acc: 0.9313 - val_loss: 1.2800 - val_acc: 0.9326
Epoch 4/10
25701/25701 [==============================] - 18s 705us/step - loss: 1.3387 - acc: 0.9432 - val_loss: 0.9766 - val_acc: 0.9138
Epoch 5/10
25701/25701 [==============================] - 17s 677us/step - loss: 1.6407 - acc: 0.9497 - val_loss: 1.2843 - val_acc: 0.9314
Epoch 6/10
25701/25701 [==============================] - 17s 676us/step - loss: 1.2290 - acc: 0.9512 - val_loss: 0.8610 - val_acc: 0.9594
Epoch 7/10
25701/25701 [==============================] - 19s 727us/step - loss: 0.8615 - acc: 0.9675 - val_loss: 0.

In [54]:
#  Policy Change Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Policy_Change, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 25.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('policy_change.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 16s 623us/step - loss: 1.9759 - acc: 0.9334 - val_loss: 0.4251 - val_acc: 0.9247
Epoch 2/10
25701/25701 [==============================] - 15s 592us/step - loss: 0.2892 - acc: 0.9683 - val_loss: 0.1446 - val_acc: 0.9661
Epoch 3/10
25701/25701 [==============================] - 16s 605us/step - loss: 0.1596 - acc: 0.9776 - val_loss: 0.1194 - val_acc: 0.9677
Epoch 4/10
25701/25701 [==============================] - 18s 685us/step - loss: 0.1295 - acc: 0.9824 - val_loss: 0.1329 - val_acc: 0.9660
Epoch 5/10
25701/25701 [==============================] - 15s 592us/step - loss: 0.1053 - acc: 0.9848 - val_loss: 0.1193 - val_acc: 0.9698
Epoch 6/10
25701/25701 [==============================] - 15s 597us/step - loss: 0.0838 - acc: 0.9880 - val_loss: 0.1215 - val_acc: 0.9660
Epoch 7/10
25701/25701 [==============================] - 15s 578us/step - loss: 0.0681 - acc: 0.9900 - val_loss: 0.

In [55]:
#  Policy Change Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Policy_Change, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 40.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('policy_change1.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 21s 804us/step - loss: 2.1168 - acc: 0.8995 - val_loss: 0.4736 - val_acc: 0.9139
Epoch 2/10
25701/25701 [==============================] - 15s 592us/step - loss: 0.3652 - acc: 0.9542 - val_loss: 0.2125 - val_acc: 0.9505
Epoch 3/10
25701/25701 [==============================] - 15s 597us/step - loss: 0.2067 - acc: 0.9704 - val_loss: 0.1493 - val_acc: 0.9611
Epoch 4/10
25701/25701 [==============================] - 15s 592us/step - loss: 0.1494 - acc: 0.9778 - val_loss: 0.1533 - val_acc: 0.9596
Epoch 5/10
25701/25701 [==============================] - 15s 593us/step - loss: 0.1089 - acc: 0.9845 - val_loss: 0.1076 - val_acc: 0.9746
Epoch 6/10
25701/25701 [==============================] - 15s 597us/step - loss: 0.0983 - acc: 0.9855 - val_loss: 0.1019 - val_acc: 0.9793
Epoch 7/10
25701/25701 [==============================] - 15s 590us/step - loss: 0.0909 - acc: 0.9869 - val_loss: 0.

In [56]:
#  Third Party Sharing/Collection Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Third_Party_Sharing_Collection, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 20}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('third_party_sharing_collection.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 16s 630us/step - loss: 2.0218 - acc: 0.3278 - val_loss: 1.1172 - val_acc: 0.3812
Epoch 2/10
25701/25701 [==============================] - 15s 592us/step - loss: 1.6558 - acc: 0.4413 - val_loss: 1.2028 - val_acc: 0.4286
Epoch 3/10
25701/25701 [==============================] - 15s 593us/step - loss: 1.4304 - acc: 0.5360 - val_loss: 0.9508 - val_acc: 0.5411
Epoch 4/10
25701/25701 [==============================] - 15s 584us/step - loss: 1.2506 - acc: 0.6167 - val_loss: 0.9378 - val_acc: 0.5754
Epoch 5/10
25701/25701 [==============================] - 16s 607us/step - loss: 1.1333 - acc: 0.6671 - val_loss: 0.9471 - val_acc: 0.6187
Epoch 6/10
25701/25701 [==============================] - 21s 833us/step - loss: 0.9833 - acc: 0.7176 - val_loss: 0.9494 - val_acc: 0.6481
Epoch 7/10
25701/25701 [==============================] - 23s 878us/step - loss: 0.8860 - acc: 0.7530 - val_loss: 0.

In [57]:
#  User Access, Edit and Deletion Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.User_Access_Edit_and_Deletion, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 20}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_access_edit_deletion.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 19s 728us/step - loss: 2.8562 - acc: 0.8621 - val_loss: 0.7851 - val_acc: 0.8636
Epoch 2/10
25701/25701 [==============================] - 17s 665us/step - loss: 0.9813 - acc: 0.8976 - val_loss: 0.7551 - val_acc: 0.8628
Epoch 3/10
25701/25701 [==============================] - 17s 664us/step - loss: 0.8291 - acc: 0.9090 - val_loss: 0.5104 - val_acc: 0.9252
Epoch 4/10
25701/25701 [==============================] - 17s 673us/step - loss: 0.7138 - acc: 0.9149 - val_loss: 0.5092 - val_acc: 0.9231
Epoch 5/10
25701/25701 [==============================] - 17s 666us/step - loss: 0.6280 - acc: 0.9228 - val_loss: 0.4813 - val_acc: 0.9142
Epoch 6/10
25701/25701 [==============================] - 17s 662us/step - loss: 0.5866 - acc: 0.9295 - val_loss: 0.4273 - val_acc: 0.9037
Epoch 7/10
25701/25701 [==============================] - 18s 707us/step - loss: 0.5629 - acc: 0.9301 - val_loss: 0.

In [58]:
#  User Access, Edit and Deletion Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.User_Access_Edit_and_Deletion, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 15.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_access_edit_deletion1.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 20s 789us/step - loss: 2.6534 - acc: 0.9083 - val_loss: 0.8942 - val_acc: 0.8509
Epoch 2/10
25701/25701 [==============================] - 19s 757us/step - loss: 0.8492 - acc: 0.9144 - val_loss: 0.4729 - val_acc: 0.9404
Epoch 3/10
25701/25701 [==============================] - 19s 756us/step - loss: 0.6887 - acc: 0.9270 - val_loss: 0.4210 - val_acc: 0.9454
Epoch 4/10
25701/25701 [==============================] - 19s 732us/step - loss: 0.5932 - acc: 0.9377 - val_loss: 0.3875 - val_acc: 0.9479
Epoch 5/10
25701/25701 [==============================] - 19s 748us/step - loss: 0.5423 - acc: 0.9412 - val_loss: 0.3680 - val_acc: 0.9242
Epoch 6/10
25701/25701 [==============================] - 19s 753us/step - loss: 0.5211 - acc: 0.9426 - val_loss: 0.3691 - val_acc: 0.9451
Epoch 7/10
25701/25701 [==============================] - 19s 743us/step - loss: 0.4991 - acc: 0.9449 - val_loss: 0.

In [59]:
#  User Choice/Control Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.User_Choice_Control, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 40.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_choice_control.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 18s 704us/step - loss: 1.9797 - acc: 0.2736 - val_loss: 1.1754 - val_acc: 0.3076
Epoch 2/10
25701/25701 [==============================] - 17s 667us/step - loss: 1.5750 - acc: 0.4779 - val_loss: 0.8585 - val_acc: 0.5046
Epoch 3/10
25701/25701 [==============================] - 17s 664us/step - loss: 1.2851 - acc: 0.6154 - val_loss: 0.7822 - val_acc: 0.6421
Epoch 4/10
25701/25701 [==============================] - 17s 663us/step - loss: 1.0296 - acc: 0.7093 - val_loss: 0.6792 - val_acc: 0.6901
Epoch 5/10
25701/25701 [==============================] - 17s 661us/step - loss: 0.8420 - acc: 0.7692 - val_loss: 0.6833 - val_acc: 0.7297
Epoch 6/10
25701/25701 [==============================] - 18s 687us/step - loss: 0.7164 - acc: 0.8126 - val_loss: 0.6601 - val_acc: 0.7611
Epoch 7/10
25701/25701 [==============================] - 18s 702us/step - loss: 0.6123 - acc: 0.8393 - val_loss: 0.

In [60]:
#  User Choice/Control Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.User_Choice_Control, test_size=0.3, random_state = 0)

vocab_length = len(embedding_matrix)

class_weights = {0: 1.,
                1: 70.}

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
                  validation_data=(x_test, y_test),class_weight=class_weights)

model.save('user_choice_control1.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

Train on 25701 samples, validate on 11016 samples
Epoch 1/10
25701/25701 [==============================] - 17s 657us/step - loss: 3.6737 - acc: 0.1501 - val_loss: 2.1342 - val_acc: 0.1993
Epoch 2/10
25701/25701 [==============================] - 14s 529us/step - loss: 3.1323 - acc: 0.2314 - val_loss: 2.4285 - val_acc: 0.2457
Epoch 3/10
25701/25701 [==============================] - 13s 519us/step - loss: 3.1131 - acc: 0.2954 - val_loss: 2.0253 - val_acc: 0.3233
Epoch 4/10
25701/25701 [==============================] - 13s 519us/step - loss: 2.8034 - acc: 0.3871 - val_loss: 1.7263 - val_acc: 0.4780
Epoch 5/10
25701/25701 [==============================] - 13s 517us/step - loss: 2.5890 - acc: 0.4751 - val_loss: 2.1495 - val_acc: 0.4223
Epoch 6/10
25701/25701 [==============================] - 13s 519us/step - loss: 2.6417 - acc: 0.5327 - val_loss: 1.9300 - val_acc: 0.4924
Epoch 7/10
25701/25701 [==============================] - 13s 516us/step - loss: 2.2864 - acc: 0.5876 - val_loss: 1.